# Kernel Density Estimation Analysis

**Notebook**: 02_kernel_density_estimation.ipynb  
**Sprint**: Phase 2 Sprint 8 - Advanced Geospatial Analysis  
**Created**: 2025-11-08  

## Objectives

1. Compute Kernel Density Estimation (KDE) for accident distribution
2. Create event density surface (all accidents equally weighted)
3. Create fatality density surface (weighted by fatalities)
4. Identify peak density locations
5. Compare event vs fatality density patterns
6. Generate interactive heatmaps

## KDE Method

- **Library**: scipy.stats.gaussian_kde
- **Bandwidth**: Scott's rule (automatic)
- **Grid Resolution**: 100x100 for US extent
- **Projections**: Both EPSG:4326 (lat/lon) and EPSG:5070 (Albers)

In [ ]:
# Standard library
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Data manipulation
import pandas as pd
import numpy as np

# Geospatial
import geopandas as gpd

# Statistics
from scipy.stats import gaussian_kde
from scipy.ndimage import maximum_filter

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Paths
DATA_DIR = Path('../../data')
FIG_DIR = Path('figures')
MAP_DIR = Path('maps')

print('✅ All packages imported successfully')

In [ ]:
# Load geospatial data
gdf = gpd.read_parquet(DATA_DIR / 'geospatial_events.parquet')
print(f'✅ Loaded {len(gdf):,} events')

# Extract coordinates and weights
coords = gdf[['dec_longitude', 'dec_latitude']].values
fatalities = gdf['inj_tot_f'].values

print(f'Coordinate range:')
print(f'  Longitude: {coords[:, 0].min():.2f} to {coords[:, 0].max():.2f}')
print(f'  Latitude: {coords[:, 1].min():.2f} to {coords[:, 1].max():.2f}')

## 1. Event Density KDE (Unweighted)

In [ ]:
# Create KDE for event density (unweighted)
print('Computing event density KDE...')
kde_event = gaussian_kde(coords.T)

# Create grid
lon_min, lon_max = coords[:, 0].min(), coords[:, 0].max()
lat_min, lat_max = coords[:, 1].min(), coords[:, 1].max()

lon_grid = np.linspace(lon_min, lon_max, 100)
lat_grid = np.linspace(lat_min, lat_max, 100)
lon_mesh, lat_mesh = np.meshgrid(lon_grid, lat_grid)

# Evaluate KDE
positions = np.vstack([lon_mesh.ravel(), lat_mesh.ravel()])
density_event = kde_event(positions).reshape(lon_mesh.shape)

print(f'✅ Event density computed')
print(f'Density range: {density_event.min():.6f} to {density_event.max():.6f}')
print(f'Grid shape: {density_event.shape}')

In [ ]:
# Visualize event density
fig, ax = plt.subplots(figsize=(14, 8))

contour = ax.contourf(lon_mesh, lat_mesh, density_event, levels=20, cmap='YlOrRd', alpha=0.7)
ax.scatter(coords[:, 0], coords[:, 1], s=0.5, c='black', alpha=0.2, label='Events')

plt.colorbar(contour, ax=ax, label='Density')
ax.set_title('Event Density Heatmap (KDE)', fontsize=14, fontweight='bold')
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(FIG_DIR / 'kde_event_density.png', dpi=150, bbox_inches='tight')
plt.show()
print('✅ Saved: kde_event_density.png')

## 2. Fatality Density KDE (Weighted)

In [ ]:
# Create weighted KDE for fatality density
print('Computing fatality density KDE...')

# For weighted KDE, replicate points based on fatality count
weighted_coords = []
for i, (lon, lat) in enumerate(coords):
    weight = int(fatalities[i]) + 1  # Add 1 to include zero-fatality events
    weighted_coords.extend([(lon, lat)] * weight)

weighted_coords = np.array(weighted_coords)
print(f'Weighted coordinates: {len(weighted_coords):,} points')

kde_fatality = gaussian_kde(weighted_coords.T)
density_fatality = kde_fatality(positions).reshape(lon_mesh.shape)

print(f'✅ Fatality density computed')
print(f'Density range: {density_fatality.min():.6f} to {density_fatality.max():.6f}')

In [ ]:
# Visualize fatality density
fig, ax = plt.subplots(figsize=(14, 8))

contour = ax.contourf(lon_mesh, lat_mesh, density_fatality, levels=20, cmap='Reds', alpha=0.7)
fatal_events = gdf[gdf['inj_tot_f'] > 0]
ax.scatter(fatal_events['dec_longitude'], fatal_events['dec_latitude'], 
           s=1, c='darkred', alpha=0.3, label='Fatal Events')

plt.colorbar(contour, ax=ax, label='Fatality-Weighted Density')
ax.set_title('Fatality Density Heatmap (Weighted KDE)', fontsize=14, fontweight='bold')
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(FIG_DIR / 'kde_fatality_density.png', dpi=150, bbox_inches='tight')
plt.show()
print('✅ Saved: kde_fatality_density.png')

## 3. Density Comparison

In [ ]:
# Side-by-side comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Event density
c1 = ax1.contourf(lon_mesh, lat_mesh, density_event, levels=15, cmap='YlOrRd', alpha=0.8)
ax1.set_title('Event Density (Unweighted)', fontsize=12, fontweight='bold')
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
plt.colorbar(c1, ax=ax1, label='Density')

# Fatality density
c2 = ax2.contourf(lon_mesh, lat_mesh, density_fatality, levels=15, cmap='Reds', alpha=0.8)
ax2.set_title('Fatality Density (Weighted)', fontsize=12, fontweight='bold')
ax2.set_xlabel('Longitude')
ax2.set_ylabel('Latitude')
plt.colorbar(c2, ax=ax2, label='Fatality-Weighted Density')

plt.tight_layout()
plt.savefig(FIG_DIR / 'kde_density_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
print('✅ Saved: kde_density_comparison.png')

## 4. Identify Density Peaks

In [ ]:
def find_density_peaks(density, lon_mesh, lat_mesh, n_peaks=10):
    """Find local maxima in density surface."""
    # Apply maximum filter to find local maxima
    local_max = maximum_filter(density, size=5) == density
    
    # Get coordinates of local maxima
    peak_indices = np.where(local_max)
    peak_values = density[peak_indices]
    
    # Get top N peaks
    top_n_idx = np.argsort(peak_values)[-n_peaks:][::-1]
    
    peaks = []
    for idx in top_n_idx:
        i, j = peak_indices[0][idx], peak_indices[1][idx]
        peaks.append({
            'lon': float(lon_mesh[i, j]),
            'lat': float(lat_mesh[i, j]),
            'density': float(density[i, j])
        })
    
    return peaks

# Find peaks
event_peaks = find_density_peaks(density_event, lon_mesh, lat_mesh, n_peaks=10)
fatality_peaks = find_density_peaks(density_fatality, lon_mesh, lat_mesh, n_peaks=10)

print('\n=== Top 10 Event Density Peaks ===')
for i, peak in enumerate(event_peaks, 1):
    print(f"{i}. Lat: {peak['lat']:.4f}, Lon: {peak['lon']:.4f}, Density: {peak['density']:.6f}")

print('\n=== Top 10 Fatality Density Peaks ===')
for i, peak in enumerate(fatality_peaks, 1):
    print(f"{i}. Lat: {peak['lat']:.4f}, Lon: {peak['lon']:.4f}, Density: {peak['density']:.6f}")

# Save peaks
peaks_data = {
    'event_density_peaks': event_peaks,
    'fatality_density_peaks': fatality_peaks
}

pd.DataFrame(event_peaks).to_csv(DATA_DIR / 'density_peaks.csv', index=False)
print('\n✅ Saved: density_peaks.csv')

## 5. Interactive Heatmaps

In [ ]:
# Event density heatmap
m_event = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

# Prepare data for HeatMap (sample for performance)
heat_data = [[row['dec_latitude'], row['dec_longitude']] 
             for idx, row in gdf.sample(min(len(gdf), 10000), random_state=42).iterrows()]

HeatMap(heat_data, radius=15, blur=25, max_zoom=13).add_to(m_event)

# Add peak markers
for i, peak in enumerate(event_peaks[:5], 1):
    folium.Marker(
        location=[peak['lat'], peak['lon']],
        popup=f"Peak {i}<br>Density: {peak['density']:.6f}",
        icon=folium.Icon(color='red', icon='fire')
    ).add_to(m_event)

# Save
m_event.save(str(MAP_DIR / 'kde_event_density.html'))
print('✅ Saved: maps/kde_event_density.html')
m_event

In [ ]:
# Fatality density heatmap
m_fatality = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

# Weight by fatalities
heat_data_fatality = [[row['dec_latitude'], row['dec_longitude'], row['inj_tot_f'] + 1] 
                      for idx, row in gdf.sample(min(len(gdf), 10000), random_state=42).iterrows()]

HeatMap(heat_data_fatality, radius=15, blur=25, max_zoom=13, gradient={0.4: 'yellow', 0.65: 'orange', 1: 'red'}).add_to(m_fatality)

# Add peak markers
for i, peak in enumerate(fatality_peaks[:5], 1):
    folium.Marker(
        location=[peak['lat'], peak['lon']],
        popup=f"Peak {i}<br>Fatality Density: {peak['density']:.6f}",
        icon=folium.Icon(color='darkred', icon='exclamation-triangle')
    ).add_to(m_fatality)

# Save
m_fatality.save(str(MAP_DIR / 'kde_fatality_density.html'))
print('✅ Saved: maps/kde_fatality_density.html')
m_fatality

## Summary

**Kernel Density Estimation Complete** ✅

**Files Created**:
- `figures/kde_event_density.png` - Event density heatmap
- `figures/kde_fatality_density.png` - Fatality density heatmap
- `figures/kde_density_comparison.png` - Side-by-side comparison
- `data/density_peaks.csv` - Top density peak locations
- `maps/kde_event_density.html` - Interactive event heatmap
- `maps/kde_fatality_density.html` - Interactive fatality heatmap

**Next Steps**:
- Getis-Ord Gi* Hotspot Analysis (03_getis_ord_gi_star.ipynb)